# Исследование надежности заемщиков

**Заказчик** — кредитный отдел банка.

**Цель:** Разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Входные данные:** статистика банка о платёжеспособности клиентов.

**Описание данных:**

`children` — количество детей в семье <br/>`days_employed` — общий трудовой стаж в днях <br/>`dob_years` — возраст клиента в годах <br/>`education` — уровень образования клиента <br/>`education_id` — идентификатор уровня образования <br/>`family_status` — семейное положение <br/>`family_status_id` — идентификатор семейного положения <br/>`gender` — пол клиента <br/>`income_type` — тип занятости <br/>`debt` — имел ли задолженность по возврату кредитов <br/>`total_income` — ежемесячный доход <br/>`purpose` — цель получения кредита

<a id="enter"></a>
**Содержание:**

1. [Открытие данных](#start)
2. [Предобработка данных](#preprocessing)
    * [2.1 Заполнение пропусков](#null)
    * [2.2 Обработка аномальных значений](#anomaly)
    * [2.3 Заполнение пропусков (продолжение)](#null1)
    * [2.4 Изменение типов данных](#types)
    * [2.5 Обработка дубликатов](#duplicates)
    * [2.6 Категоризация данных](#category)
3. [Исследование данных и ответы на вопросы](#research)
4. [Общий вывод](#conclusion)

In [1]:
#импорт библиотек
import pandas as pd

<a id="start"></a>
## Открытие данных

* Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`.

In [2]:
data = pd.read_csv('bank.csv')

* Выведем первые 10 строчек датафрейма `data` на экран.

In [3]:
data.head(10)

,Unnamed: 0,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


* Выведем основную информацию о датафрейме с помощью метода `info()`.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        21525 non-null  int64  
 1   children          21525 non-null  int64  
 2   days_employed     19351 non-null  float64
 3   dob_years         21525 non-null  int64  
 4   education         21525 non-null  object 
 5   education_id      21525 non-null  int64  
 6   family_status     21525 non-null  object 
 7   family_status_id  21525 non-null  int64  
 8   gender            21525 non-null  object 
 9   income_type       21525 non-null  object 
 10  debt              21525 non-null  int64  
 11  total_income      19351 non-null  float64
 12  purpose           21525 non-null  object 
dtypes: float64(2), int64(6), object(5)
memory usage: 2.1+ MB


Итак таблица состоит из 21525 строк и 12 столбцов.

- 4 столбца имеют формат `int` (целые числа):
<br/>* `children`(количество детей), <br/>* `dob_years`(возраст заемщика), <br/>* `education_id` (идентификатор уровня образования), <br/>* `family_status_id` (семейный статус), <br/>* `debt`(должник или нет)

- 2 столбца имеют формат `float` (вещественные числа):
<br/>* `days_employed`(дней трудового стажа), <br/>* `total_income`(ежемесячный доход)
- остальные столбцы имеют формат `object` (общий формат, который может включать текст, символы или числа)

[к содержанию](#enter)

<a id="preprocessing"></a>
## Предобработка данных

<a id="null"></a>
### Заполнение пропусков

Существуют разные причины появления пропусков:
- пользователь случайно или умышленно указал не все данные;
- в системе сбора информации возникли неполадки.
 Бывает так что просто нет значения и бывает что есть специальные указатели вместо пропусков None (тип NoneType) и NaN (тип float).
 В данном случае имеется одинаковое количество пропусков в столбцах занятости и дохода, что врят ли можно расценивать как случайное совпадение. Поэтому наиболее достоверной причиной мне кажется отсутствие работы, и соответственно, источника дохода.

* Выведем количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

Unnamed: 0             0
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

[к содержанию](#enter)

<a id="anomaly"></a>
### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получаются аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Тем более этот столбец не понадобится для исследования.

Выведем перечень уникальных значений столбца `children`.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

[к содержанию](#enter)

<a id="null1"></a>
### Заполнение пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждому типу занятости `income_type`.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены:

In [13]:
data.isna().sum()

Unnamed: 0          0
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

[к содержанию](#enter)

<a id="types"></a>
### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [14]:
data['total_income'] = data['total_income'].astype(int)

[к содержанию](#enter)

<a id="duplicates"></a>
### Обработка дубликатов

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [15]:
data.duplicated().sum()

0

In [16]:
data = data.drop_duplicates()

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. 

In [17]:
data['education'] = data['education'].str.lower()

[к содержанию](#enter)

<a id="category"></a>
### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


*Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используем собственную функцию с именем `categorize_income()` и метод `apply()`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

*Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

[к содержанию](#enter)

<a id="research"></a>
## Исследование данных и ответы на вопросы

**Исследование зависимости между количеством детей и возвратом кредита в срок**

In [23]:
data_grouped=data.groupby('children').agg({'debt':'sum', 'children': 'count'})
data_grouped['debt_rate']=data_grouped['debt']/data_grouped['children']
print(data_grouped)

          debt  children  debt_rate
children                           
0         1063     14149   0.075129
1          444      4818   0.092154
2          194      2055   0.094404
3           27       330   0.081818
4            4        41   0.097561
5            0         9   0.000000


*Комментарий: Интересно то, что с увеличением количества детей меньше людей связывают себя кредитами. Подавляющее большинство заемщиков детей не имеют.*

**Вывод:** Очевидно, что если относиться буквально и рассмотреть все категории, явной зависимости количества должников от количества детей не прослеживается. Среди бездетных уровень должников наименьший - 7,5 %, больше всего доля среди должников которые имеют 4х детей - 9,7 %, а среди тех у кого 5 детей - должников вообще нет. Должники с 1 ребенком составляют 9,2 %,  с двумя - 9,5 %, с тремя - 8,2 %.
Тем не менее, основываясь на максимальной выборке и рассмотрев первые 3 строчки тендеция есть, чем больше детей, тем больше должников.

**Исследование зависимости между семейным положением и возвратом кредита в срок**

In [24]:
data_grouped=data.groupby('family_status_id').agg({'debt':'sum', 'family_status_id': 'count'})
data_grouped['debt_rate']=data_grouped['debt']/data_grouped['family_status_id']
data_grouped['family_status']=data['family_status'].unique()
print(data_grouped)

                  debt  family_status_id  debt_rate          family_status
family_status_id                                                          
0                  927             12302   0.075354        женат / замужем
1                  385              4160   0.092548       гражданский брак
2                   63               952   0.066176         вдовец / вдова
3                   84              1189   0.070648              в разводе
4                  273              2799   0.097535  Не женат / не замужем


*Комментарий: Как видно чаще всего пользуются кредитами клиенты состоящие в официальном браке, реже вдовы/вдовцы. Конечно, у семьи потребности выше чем у тех кто не связан узами брака (необходимость в постоянном жилье, собственном автомобиле, образовании и т.д.). Эта категория составляет основную долю дохода от кредитования физических лиц и обладает средним кредитным риском. Существенных отличий количества должников от семейного статуса нет. Больше всего среди холостяков никогда не вступавших в брак, чуть меньше из тех кто в неофициальном браке, ещё меньше среди разведенных и наименьшая доля должников среди вдов/вдовцов. 

**Вывод:** Результаты показали, что в зависимости от семейного статуса колебание по уровню должников относительно количества заемщиков по каждой группе составляет порядка 3 %. Категория вдовцов имеет минимум должников - 6,6 %, чуть выше показатель у разведенных - 7 %, за ними идут семейные - 7,6%, затем состоящие в гражданском браке - 9,3% и потом холостяки - с максимальным показателем - 9,8 %.

**Исследование зависимости между уровнем дохода и возвратом кредита в срок**

In [25]:
data_grouped=data.groupby('total_income_category').agg({'debt':'sum', 'total_income_category': 'count'})
data_grouped['debt_rate']=data_grouped['debt']/data_grouped['total_income_category']
print(data_grouped)

                       debt  total_income_category  debt_rate
total_income_category                                        
A                         2                     25   0.080000
B                       354                   5014   0.070602
C                      1353                  15992   0.084605
D                        21                    349   0.060172
E                         2                     22   0.090909


*Комментарий: Что характерно и, очевидно, не случайно большинство заемщиков находятся в категории С (доход от 50000 - 200000). Категории В и выше видимо меньше нуждаются в предлагаемом займе, а по крупным займам для физических лиц существуют ограничения. Категории Е и D наверное не только не могут себе позволить жить в долг, но и реже получают одобрение в банке.

**Вывод:** Результаты показали, что относительный уровень должников в зависимости от дохода имеет небольшие колебания(порядка 3%). Категория Е, где доход < 30000, имеет наибольшую долю должников (9 %), чуть ниже доля заемщиков в категории С - 8,5 %, далее категория А - 8 %, потом В - 7,1 %. Наименьшая доля должников в категории D - 6 %.

**Исследование зависимости цели кредита на его возврат в срок**

In [26]:
data_grouped=data.groupby('purpose_category').agg({'debt':'sum', 'purpose_category': 'count'})
data_grouped['debt_rate']=data_grouped['debt']/data_grouped['purpose_category']

print(data_grouped)

                          debt  purpose_category  debt_rate
purpose_category                                           
операции с автомобилем     400              4288   0.093284
операции с недвижимостью   780             10780   0.072356
получение образования      369              3997   0.092319
проведение свадьбы         183              2337   0.078306


**Вывод:** Очевидно, что основная категория займа - операции с недвижимостью, поскольку она является самой надежной, наименее рискованной - 7,3 %. Немного больше должников за свадебные церемонии - 7,9 %.  Операции с автомобилем и образование - имеют наибольший риск невозврата ~ 9,3% (из-за многих факторов: аварии, потеря работы, снижение себестоимости авто, инфляция, сложности трудоустройства, форсмажор и т.д.).

<a id="conclusion"></a>
## Общий вывод.

Разумеется, полученная картина - результат не только спроса граждан, но и конкретной политики банка и для комплексной оценки надежности системы кредитования этой информации не вполне достаточно.
Тем не менее, на основе предоставленных данных можно сделать следущие выводы:
- Наиболее надежные заемщики - находящиееся в официальном браке и не имеющие детей с уровнем дохода 30000-50000;
- Самые надежные цели займа - операции с недвижимостью.


[к содержанию](#enter)